# 1. RAG chain 구현 구문

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

# 평가 알로리즘 모듈
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field


# 메모리 관련 모듈
from langchain_core.chat_history import InMemoryChatMessageHistory

from langchain.chains import RetrievalQA
from langchain.schema import AIMessage, HumanMessage




from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()


True

In [3]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 저장된 데이터 내용 확인
documents = vector_store._collection.get()['documents']
metadatas = vector_store._collection.get()['metadatas']

print(f"Documents: {documents[:5]}") 
print(f"Metadatas: {metadatas[:5]}")

Documents: ['foodDetailTypes: 스시\nheaderInfo_nameKR: 스시조\nheaderInfo_nameEN: Sushi Cho\nheaderInfo_nameCN: \nheaderInfo_bookYear: 2025\nheaderInfo_ribbonType: 3\ndefaultInfo_chefName: \ndefaultInfo_phone: 02-317-0373\ndefaultInfo_openHours: \ndefaultInfo_closeHours: \ndefaultInfo_openHoursWeekend: \ndefaultInfo_closeHoursWeekend: \ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: False\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: 웨스틴조선호텔 20층\nstatusInfo_menu: 런치(Hall)(1인 15만5천원~20만5천원), 디너(Hall)(1인 19만4천원~33만원), 스시조회덮밥(10만원, 프리미엄 13만원), 복가라아게돌솥밥(12만원), 굴돌솥밥(7만8천원), 활새우튀김(11만원), 조리장특선모둠스시(14만5천원)\nstatusInfo_priceRange: 25만원 이상\nstatusInfo_openDate: 2008년\nstatusInfo_businessHours: 12:00~15:00/17:30~22:00(마지막 주문 21:30)\njuso_detailAddress: 웨스틴조선호텔 20층\njuso_roadAddrPart1: 서울특별시 중구 소공로 106\njuso_engAddr: 106, Sogong-ro, Jung-gu, Seoul\njuso_bdNm: 서울 웨스틴조선호텔\njuso_siNm: 서울특별시\njuso_sggNm: 중구\njuso_emdNm: 소공동\njuso_liNm: \njuso_rn: 소공로\njuso_buldMnnm: 106\njuso_buldSln

In [13]:
QUERY = " 강남구 근처에 가족이나 연인과 함께 가기 좋은 음식점 세 곳만 추천해줘"  


# embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)


vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)


# Retriever 생성 - "Map Reduce" 방식 - 더 정확한 답변
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10, "lambda_mult":0.5}
)


# Prompt Template 생성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True,  # Include source documents in response
)


response = retrieval_qa({"query": QUERY})

print("응답:", response["result"])

응답: 강남구 근처에 가족이나 연인과 함께 가기 좋은 음식점 세 곳은 다음과 같습니다:

1. **고메고메**
   - 주소: 서울특별시 강남구 삼성로85길 38
   - 전화: 02-555-9567
   - 특징: 참숯을 넣은 항아리에 숙성한 고기를 구워 먹는 곳으로, 모던한 인테리어와 함께 다양한 고기 메뉴를 즐길 수 있습니다.

2. **상하이**
   - 주소: 서울특별시 강남구 논현로 508 (GS강남타워 B1층)
   - 전화: 02-2005-1003
   - 특징: 1930년대 상하이의 고급 레스토랑 분위기를 콘셉트로 하며, 광동식과 매콤한 사천식 요리를 주로 선보입니다. 고급스러운 분위기에서 중식을 즐길 수 있습니다.

3. **소호정**
   - 주소: 서울특별시 서초구 논현로 27
   - 전화: 02-579-7282
   - 특징: 경상도 안동 지방의 반가 음식인 안동 국시를 맛볼 수 있는 곳으로, 다양한 한식 메뉴가 있어 가족과 함께 가기 좋습니다.

이 세 곳은 모두 가족이나 연인과 함께 즐기기에 적합한 분위기와 메뉴를 제공합니다.


In [ ]:
#########################################################
# InMemoryVectorStore 구문 추가 코드 - 미완성
#########################################################
QUERY = " 강남구 근처에 가족이나 연인과 함께 가기 좋은 음식점 세 곳만 추천해줘"  


# embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)


vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)


# Retriever 생성 - "Map Reduce" 방식 - 더 정확한 답변
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10, "lambda_mult":0.5}
)


# Prompt Template 생성
prompt_template = ChatPromptTemplate(
    [
        ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
        MessagesPlaceholder("history"), 
        ("human", "{query}")
    ]
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True,  # Include source documents in response
)


response = retrieval_qa({"query": QUERY})

print("응답:", response["result"])

<class 'list'> 1536


In [16]:
documents = retriever.get_relevant_documents("리본 두개 이상인 서울 한식집을 알려주세요.")
print(f"Retrieved {len(documents)} documents")
for doc in documents:
    print(doc.page_content)

Retrieved 5 documents
id: 6364
createdDate: 1304147828000
bookStatus: APPROVAL
chefName: 박승재
closeHours: nan
closeHoursWeekend: nan
dayOff: 일, 월, 화요일 휴무
openHours: nan
openHoursWeekend: nan
phone: 010-5538-3973
website: nan
gps_latitude: 37.552014
gps_longitude: 126.927861
ribbonType: nan
juso_detailAddress: nan
juso_roadAddrPart1: 서울특별시 마포구 와우산로30길 80
review_text: 한식 주점의 정석과도 같은 곳. 손맛 좋은 오너 겸 셰프가 내는 한식을 안주로 삼아 저녁때 술 한잔 즐기기 좋다. 소갈비찜과 스팸골뱅이는 빼놓지 말고 주문해야 하는 메뉴. 그날의 신선한 재료로 만드는 요리도 매번 기대하게 하는 메뉴다.
review_simple: 한식 주점의 정석과도 같은 곳. 손맛 좋은 오너 겸 셰프가 내는 한식을 안주로 삼아 저녁때 술 한잔 즐기기 좋다. 소갈비찜과 스팸골뱅이는 빼놓지 말고 주문해야 하는 메뉴. 그날의 신선한 재료로 만드는 요리도 매번 기대하게 하는 메뉴다.
statusInfo_businessHours: 18:00~23:00(마지막 주문 22:00) 
statusInfo_creditCard: y
statusInfo_menu: 미로소갈비찜(3만2천원), 닭튀김(2만원), 양념돼지목살구이(2만5천원), 골뱅이무침과스팸구이(2만2천원), 해물부추전, 애호박감자채전(각 1만8천원)
foodType_1: 한식(일반한식)
foodDetailType_1: 모던한식
foodType_2: 모던한식
foodDetailType_2: 한식주점
foodType_3: 한식주점
foodDetailType_3: nan
foodType_4: nan
foodDetailType_4: nan
foodType_5: 

In [17]:
for doc in docs:
    print("Context:", doc.page_content)
    print("Question:", question)

NameError: name 'docs' is not defined

#### Map_reduce 확인

In [14]:
r = map_reduce_chain.invoke("리본 두개 이상인 서울 한식집을 알려주세요.")
print(r)

''

''

''

''

''




#### 최종 답변 

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = ({"context":map_reduce_chain, "question":RunnablePassthrough()} 
        | final_prompt
        | model
        | StrOutputParser())

In [ ]:
#

## 기록

```python
# # Retriever 생성 - 성능이 안좋음: 부정확한 답변
# retriever = vector_store.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 5}, 
# )

```

## 데이터 검토를 위한 코드들

In [15]:
# Retriever를 통해 관련 문서 가져오기
documents = retriever.get_relevant_documents(QUERY)

# 관련 문서 출력
for idx, doc in enumerate(documents):
    print(f"문서 {idx + 1}:")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)
    print("------")

문서 1:
Metadata: {}
Content: id: 4091
createdDate: 1216620773000
bookStatus: INACTIVE
chefName: nan
closeHours: nan
closeHoursWeekend: nan
dayOff: 명절 휴무
openHours: nan
openHoursWeekend: nan
phone: 02-307-9979
website: nan
gps_latitude: 37.57189
gps_longitude: 126.934353
ribbonType: nan
juso_detailAddress: nan
juso_roadAddrPart1: 서울특별시 서대문구 연희로25길 31-3
review_text: 가정집을 개조한 아늑한 분위기의 한정식집. 떡갈비와 간장게장이 인기 메뉴며 반찬도 정갈하다. 가정집에 초대받아 음식을 대접받는 기분으로 식사를 즐길 수 있다. 가격 대비 만족도도 높은 편이다.
review_simple: 가정집을 개조한 아늑한 분위기의 한정식집. 떡갈비와 간장게장이 인기 메뉴며 반찬도 정갈하다. 가정집에 초대받아 음식을 대접받는 기분으로 식사를 즐길 수 있다. 가격 대비 만족도도 높은 편이다.
statusInfo_businessHours: 11:30~15:00/17:00~21:00 
statusInfo_creditCard: y
statusInfo_menu: 떡갈비정식(2만1천원), 간장게장정식(3만8천원), 보리굴비정식(2만6천원), 수빈정식세트(4인 7만8천원~10만9천원)
foodType_1: 한식(일반한식)
foodDetailType_1: 한정식
foodType_2: 한정식
foodDetailType_2: 게장
foodType_3: 한식(어패류)
foodDetailType_3: nan
foodType_4: 게장
foodDetailType_4: nan
foodType_5: nan
foodDetailType_5: nan
foodType_6: nan
foodDetailType_6: nan
foodTyp